In [ ]:
import os, sys
import urllib, cStringIO

import pymongo as pm

import matplotlib
from matplotlib import pylab, mlab, pyplot
%matplotlib inline
from IPython.core.pylabtools import figsize, getfigs
plt = pyplot
import seaborn as sns
sns.set_context('poster')
sns.set_style('white')

import numpy as np
from __future__ import division
import scipy.stats as stats
import pandas as pd
import json
import re

from IPython.display import clear_output

from PIL import Image
import base64

In [ ]:
# directory & file hierarchy
proj_dir = os.path.abspath('..')
analysis_dir = os.getcwd()
results_dir = os.path.join(proj_dir,'results')
plot_dir = os.path.join(results_dir,'plots')
csv_dir = os.path.join(results_dir,'csv')
exp_dir = os.path.abspath(os.path.join(proj_dir,'experiments'))
sketch_dir = os.path.abspath(os.path.join(proj_dir,'sketches'))
feature_dir = os.path.abspath(os.path.join(proj_dir,'features'))
svg_dir = os.path.abspath(os.path.join(sketch_dir,'svg'))
png_dir = os.path.abspath(os.path.join(sketch_dir,'png'))

## add helpers to python path
if os.path.join(proj_dir,'analysis') not in sys.path:
    sys.path.append(os.path.join(proj_dir,'python'))
    
if not os.path.exists(results_dir):
    os.makedirs(results_dir)
    
if not os.path.exists(plot_dir):
    os.makedirs(plot_dir)   
    
if not os.path.exists(csv_dir):
    os.makedirs(csv_dir)       
    
# Assign variables within imported analysis helpers
import analysis_helpers as h
import rendering_helpers as srh
if sys.version_info[0]>=3:
    from importlib import reload
reload(h)
reload(srh)

In [ ]:
# path to npz files
path_npz = os.path.join(sketch_dir,'quickdraw','npz')
path_svg = os.path.join(sketch_dir,'quickdraw','svg')
path_png = os.path.join(sketch_dir,'quickdraw','png')

# how many drawings from each category to include
sample_size = 5000

# list of all sketch classes
class_list = ['bear','cat','duck','lion','pig','rabbit','sheep','swan']

# sorted list of all categories
paths = [os.path.join(path_npz,i) for i in sorted(os.listdir(path_npz))]

In [ ]:
# render params
padding = 10 # how much padding 
stroke_pct_canvas = 0.02 ## what fraction of image size should a stroke's width be?

# loop through paths and render out
for path in paths:
    category = path.split('/')[-1].split('.')[0]
    x = np.load(path)
    y = x['train']
    for i in np.arange(sample_size):    
        lines = srh.strokes_to_lines(y[i])
        this_svg, bounds = srh.make_svg_list(lines)

        ## render to svg file
        stroke_width = np.int(np.round(stroke_pct_canvas*bounds))
        out_dir = os.path.join(path_svg,category)
        out_fname = '{:04d}'.format(i)
        if not os.path.exists(os.path.join(path_svg,category)):
            os.makedirs(os.path.join(path_svg,category))
        srh.render_svg(this_svg,
                     out_dir=out_dir,
                     viewbox=[0, 0, bounds+padding, bounds+padding],
                     stroke_width = stroke_width,
                     stroke_color = 'black',
                     out_fname= '{}.svg'.format(out_fname)) 
        print('Rendering {}/{}'.format(out_dir,out_fname))
        clear_output(wait=True)

In [ ]:
## get svg path list for rendered out svg
svg_paths = srh.generate_svg_path_list(os.path.join(path_svg))    

## convert all svg to png
for svg_path in svg_paths:
    category = svg_path.split('/')[-2]
    srh.svg_to_png([svg_path],base_dir=path_png,out_dir=category)    

In [ ]:
## extract quickdraw features using VGG fc6 (this takes a few minutes)
for this_category in class_list:
    print 'Extracting quickdraw features for {}'.format(this_category)
    clear_output(wait=True)
    cmd_string = "python extract_features.py --data='/home/jefan/collabdraw/sketches/quickdraw/png/{}' --layer_ind=5 --data_type='sketch' --spatial_avg=True --channel_norm=False --out_dir='/home/jefan/collabdraw/features/quickdraw/{}'".format(this_category,this_category)
    os.system(cmd_string)
print 'Done!'